In [ ]:
# step 1 run this cell to import the library and access the gpt api
import json
import re
import faiss
import torch
import requests
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel
import re
# ----------------------- SETUP -----------------------
API_KEY = ""
ENDPOINT = ""
headers = {
    "Content-Type": "application/json",
    "api-key": API_KEY,
}


In [26]:
# step3 to call the gpt api
def get_response(prompt):
    headers = {
        "Content-Type": "application/json",
        "api-key": API_KEY,
    }
    payload = {
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 1600
    }
    try:
        response = requests.post(ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()
        return response.json().get("choices", [{}])[0].get("message", {}).get("content", "")
    except Exception as e:
        return f"Error: {e}"


In [54]:
respone = get_response('i want ot go to naintial with my family for a vacation, suggest some agents who can help me book my trip. which local search keyword should I search on justdial for this query')

In [55]:
print(respone)

For planning a trip to Nainital with your family, you can search for travel agents or tour operators who specialize in family vacations or hill station trips. On Justdial, you can use the following local search keywords to find suitable agents:

1. "Travel agents for Nainital"
2. "Tour operators for Nainital"
3. "Family vacation packages Nainital"
4. "Nainital trip planner"
5. "Nainital holiday packages"
6. "Nainital travel agents"
7. "Nainital tourism services"

By using these keywords, you should be able to find several travel agents and tour operators who can assist you in planning your trip to Nainital. Make sure to check reviews and compare packages to find the best option for your family.


In [8]:
# step4 prompt 1
def prompt1(history_conversation , original_input):
    history_text = ""
    for i, (question, reply) in enumerate(history_conversation.items(), start=1):
        history_text += f"{i}. You asked: {question}\n   User replied: {reply}\n"
    # print(history_text)
    prompt2 = f"""
    You are an intelligent assistant helping Indian users find the right local service or business. Your job is to extract details from the user’s query in the exact sequence:
    User who initially entered this query: "{original_input}"
    The following clarifications have already taken place:{history_text}
    Follow the steps in the sequential manner. Do not fill the value of one field into another, e.g., do not use area to infer city. If something is missing, ask the related clarification question until it's confirmed, and then proceed to the next step.
    you must accurately extract three details:
    -Step 1 – Category Extraction (LLM Instruction):
    Extract the most suitable and generalized service, product, or business type from the user query.
    Normalize the query: convert Hindi, Hinglish, or misspelled English into correct English.
    Extracted category should be generalizing the category so it represents the broad need instead of a narrow subtype.
    Expand the output into a clear explanatory phrase of at least five words. Example: “AC Repair Shop (where air conditioners get repaired)”.
    If the query implies a category (e.g., “I want to travel to Mumbai”), deduce the most probable service types (bus booking, train tickets, flights, taxis) and select the closest generalized match instead of open-ended questioning.
    If ambiguity remains, ask only one clarifying question with limited options (max 3–4).
    If the query cannot be mapped to any valid service/business type, politely request the user to rephrase or provide more context.
    Do not proceed to Step 2 until Category is confirmed.
    After confirmation please also confirm the city and area.
    -Step 2 – City Extraction (Second Priority): The city in India where the service is required. If no city is provided, ask for it.
    If travel-related query like 'I want to travel to Mumbai', ask 'From which city do you want to travel to Mumbai?'.
    If multiple cities are mentioned:
      For travel queries → determine starting city like if user query is I want to travel to kanpur from mumbai then city shoulb be mumbai;
      For service queries → determine the service location city in which user want the service.
    If the location is a locality that can be confidently mapped to a city, confirm with the user.
    If the city is outside India, respond: 'Sorry for the inconvenience, but our services are currently available only in India.
    Could you please share the name of an Indian city where you would like to request this service? I can help you with that.'
    -Do not proceed to Step 3 until City is confirmed.
    Step 3 – Area Extraction (Final Priority):
    The locality, neighbourhood, colony, or smaller part of the city where they want the service. If only a city is provided, ask 'Which area in <city> are you searching in?'.
    The area should not be the same as the city.
    When the extracted area does not exist in the given city, use a similarity check to find and return the closest matching area from that city
    If the area is written in Hindi, Hinglish, or misspelled English, convert it to correct English.
    If area seems invalid or mismatched, ask the user to provide a correct locality.
    - If all three details (category, area, city) are confidently identified and specific and there is no missing value and all value is filled  then only return in this format:
      Return only this JSON format (no explanation text):
          {{"category": "category_name", "area": "area_name", "city": "city_name"}}
    If any detail is missing or unclear, output a direct question to get that detail.
    Examples: 'Are you looking for a train ticket agent, bus booking, or flight booking?', 'Could you please specify the city where you need this service?', 'Which area in Delhi are you searching in?'.
    Special Handling / Edge Cases:
    Travel Queries → Always ask both origin and destination if unclear, plus service type. Event-based Queries (e.g., 'Doctor near me') → Ask for both city and area.
    Multiple Locations → Determine service city vs. starting point (travel).
    Non-service Queries → Politely ask them to rephrase in terms of a local business/service. """
    return prompt2

In [9]:
#step5 Prompt 2 to handle the threshold
def prompt2(history_conversation, cat, area_query, threshold_cat, threshold_area, cat1, area):
    # Build history text from past conversation
    history_text = ""
    for i, (question, reply) in enumerate(history_conversation.items(), start=1):
        history_text += f"{i}. You asked: {question}\n   User replied: {reply}\n"

    # Build cat_text and area_text based on the categories and areas provided
    cat_text = ""
    for i, category in enumerate(cat1, start=1):
        cat_text += f"{i}. {category}\n"

    area_text = ""
    for i, location in enumerate(area, start=1):
        area_text += f"{i}. {location}\n"

    # Construct the main prompt
    prompt2 = f"""
You are assisting a user in identifying a local business category and area based on their query.
Here is the conversation history:
{history_text}
From the initial understanding:
- **Category**: "{cat}"
- **Area**: "{area_query}"
After comparing with our internal database using FAISS similarity search:
- **Category similarity score**: {threshold_cat}
- **Area similarity score**: {threshold_area}

The threshold for category or area is below our confidence level of 0.8 or 0.9 , indicating that the provided information may be **vague**, **uncommon**, **misspelled**, or **ambiguous**.
Action Required:
- If **Category similarity score** score is below 0.8: Provide the top similar categories from the database, specifically using the `cat1` column (which contains potential categories from the database) and ask for clarification. Here are some similar categories:
{cat_text}

- If **Area similarity score** score is below 0.9: Provide the top similar areas from the database, specifically using the `area` column (which contains potential areas from the database) and ask for clarification. Here are some similar areas:
{area_text}

- Prioritize the item with the **lower threshold** (either category or area) for clarification.

Your task:
Generate **one simple follow-up question** based on the following:
- If the category is unclear: "Are you looking for [specific example of categories from {cat_text}]?"
- If the area is unclear: "Could you please tell me which locality or area within {area_text} you're searching in?"
- If both are unclear: "Could you clarify both the exact service you're looking for and the area within your city?"

Special Cases:
- Multiple locations mentioned: Ask the user to clarify the service location vs. starting point if related to travel.
- Ambiguous categories (e.g., “repair”, “booking”, “printing”): Ask for more specifics on the type.
- Non-service-related queries: Politely ask the user to rephrase or clarify their needs.

Return only the **follow-up question**. Do not include explanations or additional information.
"""
    return prompt2


In [10]:
# step6
def prompt_follow1(categories , cat_org):
    category_text = ""
    for cat in categories :
        category_text += f"{cat} , "
    # print(history_text)
    prompt2 = f"""
    You are an intelligent assistant helping Indian users find the right local service or business.
    This is the {cat_org} which is extracted from the usery queries.
    These are the Link categories {category_text}
    Your Task to ask the follow up question regarding the Link category.
    Example Are you also interesting in {category_text}?
    Use the Example as context do not return the question which look like same as the example.
    Give the follow up question grammarily correct . and simple ensure Link categories be in the question.
     """
    return prompt2

In [11]:
pip install pyarrow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [22]:
# pip install fastparquet

   ---------------------------------------- 0.0/673.3 kB ? eta -:--:--
   ---------------------------------------- 673.3/673.3 kB 7.5 MB/s  0:00:00
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 10.5 MB/s  0:00:00

   -------------------- ------------------- 1/2 [fastparquet]
   ---------------------------------------- 2/2 [fastparquet]

Note: you may need to restart the kernel to use updated packages.


In [12]:
# search_runtime.py
# ensure to install transformers torch faiss-cpu pandas numpy pyarrow
# To import the embeddeing model
# step7

# Change the path of directory.
# ART_DIR = "/content/drive/MyDrive/justdial for whatsapp"
MODEL_NAME = "intfloat/multilingual-e5-base"

# Load metadata
# df_cat = pd.read_parquet(f"df_cat.parquet")
df_cat = pd.read_parquet("df_cat.parquet", engine="fastparquet")
df_area = pd.read_parquet("df_area1.parquet" , engine="fastparquet")

# Load FAISS indexes
index_cat = faiss.read_index(f"index_cat.faiss")
index_area = faiss.read_index(f"index_area1.faiss")

# Load model for query encoding
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tok = AutoTokenizer.from_pretrained(MODEL_NAME)
mdl = AutoModel.from_pretrained(MODEL_NAME).to(device).eval()

@torch.inference_mode()
def e5_encode(texts, is_query=False, batch_size=64, max_length=64):
    prefix = "query: " if is_query else "passage: "
    vecs = []
    for i in range(0, len(texts), batch_size):
        batch = [prefix + str(t).strip() for t in texts[i:i+batch_size]]
        enc = tok(batch, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to(device)
        out = mdl(**enc).last_hidden_state[:, 0]
        out = torch.nn.functional.normalize(out, p=2, dim=1)
        vecs.append(out.cpu().numpy().astype("float32"))
    return np.vstack(vecs)

In [37]:
def format_category(text):
    # Split into main category and explanation
    if "(" in text and ")" in text:
        main, explanation = text.split("(", 1)
        main = main.strip()
        explanation = "(" + explanation.strip()
        return f"Main category is {main} and this is category explanation {explanation} "
    else:
        # If no explanation given
        return f"Main category is {text.strip()}"
# Example
# category = "Tent Rental Services (where tents are rented for events)"
# output = format_category(category)
# print(output)


In [78]:
# step 8
# Search algorithm for the embedding model
def _safe_cols(df: pd.DataFrame, wanted_cols: list) -> list:
    return [c for c in wanted_cols if c in df.columns]

def search_categories(user_query: str, top_k: int = 5) -> pd.DataFrame:
    user_query = format_category(user_query)
    qv = e5_encode([user_query], is_query=True)
    D, I = index_cat.search(qv, top_k)
    res = df_cat1.iloc[I[0]].copy()
    res["cosine_similarity"] = D[0]
    cols = _safe_cols(res, ['national_catid', 'categroy_name' , 'embedding'])
    return res[cols + ["cosine_similarity"]] , qv

def search_areas(user_query: str, top_k: int = 5) -> pd.DataFrame:
    user_query = user_query.lower()
    cleaned = re.sub(r"[^\w]", "", user_query)
    qv = e5_encode([cleaned], is_query=True)
    D, I = index_area.search(qv, top_k)
    res = df_area.iloc[I[0]].copy()
    res["cosine_similarity"] = D[0]
    cols = _safe_cols(res, ["main_area", "pincode", "city"])
    return res[cols + ["cosine_similarity"]]

In [69]:
df_cat1 = pd.read_pickle('df_cat_with_embeddings.pkl')

In [72]:
df_cat1.head()

,national_catid,categroy_name,embedding
0,10050521,Body Massage Centres,"[0.020193165, -0.011742339, -0.016831696, -0.0..."
1,11010608,Massage Centres For Men,"[0.019172434, -0.0006042592, -0.029820744, -0...."
2,10272436,Interior Designers,"[-0.005018342, 0.014741824, -0.023653127, 0.00..."
3,10076456,Car Rental,"[0.026977241, 0.008907894, -0.009950297, 0.018..."
4,10042600,Beauty Spas,"[0.015548817, -0.0037554502, -0.023473088, 0.0..."


In [89]:
df , qv = search_categories('chemists')

In [90]:
df.head()

,national_catid,categroy_name,embedding,cosine_similarity
1950,10502663,Tutorials For Class XII Chemistry,"[-0.016723225, 0.023238415, -0.022944383, 0.03...",0.884532
262,10096237,Chemists,"[-0.016057754, 0.01176591, -0.030365767, 0.001...",0.883691
3685,10623805,Tutorials For Class XI Chemistry,"[-0.018147292, 0.024671588, -0.025833273, 0.02...",0.881250
1392,10502611,Chemistry Tutorials,"[-0.006465669, 0.003201086, -0.019049563, 0.02...",0.879046
6764,11575237,Tutorials For Neet Chemistry,"[-0.015928844, 0.01483429, -0.012638513, 0.016...",0.874187


In [15]:
# step 9
def extract_json(response):
    match = re.search(r'\{.*?\}', response, re.DOTALL)
    if match:
        try:
            return json.loads(match.group())
        except:
            return None
    return None


In [16]:
#  step 10

def extract_category(text):
    # Use a regex pattern to capture everything before the first '('
    match = re.match(r"^([^(]+)", text)
    if match:
        return match.group(1).strip()
    else:
        return None  # Return None if no match is found

  # Output: Gym Service


In [17]:
  # step 11
  not_field = {
    "N/A", "NA", "n/a", "na", "null", "NULL", "", "-", "none", "None",
    "not available", "not applicable", "unspecified", "unknown",
    "not_available", "not_applicable", "not_provided", "not_supplied",
    "not_captured", "not_recorded", "not_collected", "not_entered",
    "not_disclosed", "not_shared", "not_furnished", "not_known",
    "not_identified", "not_determined", "undetermined", "missing",
    "missing_value", "missing_data", "no_data", "no_response",
    "no_answer", "no_result", "empty_value", "blank_value",
    "null_value", "withheld", "redacted", "confidential",
    "pii_removed", "policy_restricted", "compliance_restricted",
    "restricted", "forbidden", "blocked", "safety_blocked",
    "content_blocked", "ambiguous", "unclear", "vague",
    "needs_clarification", "insufficient_context", "insufficient_detail",
    "low_confidence","not provided"
        }

In [18]:
def preprocess_text(text):
    if not text:
        return ""
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)  # keep only letters, numbers, spaces
    return text.strip()

In [19]:
# step 12
# main function handle all conversation and follow up question
def main_function(response, area_1="", city_1="" , start=False):
    first_user = input(f"Assitant : {response}\nUser : ")
    history_conversation = {}
    a = response
    a = a.lower()
    if start == True:
      first_user +=f"area is {area_1} , and city is {city_1} Please used this only as area and city not ask the question related area and city but ask question on categories for clarity"
    history_conversation[a] = first_user.lower()
    threshold_cat = 0
    threshold_area = 0
    cnt = 0
    df_cat_res = pd.DataFrame()
    df_area_res = pd.DataFrame()
    cat = ''
    area = ''
    city = ''
    # Adjusted threshold condition for better clarity
    while ((threshold_cat < 0.7 or threshold_area < 0.75) and cnt < 7):
        # Assuming prompt_fun1 generates the query for extracting data
        prompt2 = prompt1(history_conversation, first_user)
        response = get_response(prompt1)
        parsed = extract_json(response)

        if parsed:
            cat = parsed.get("category")
            area = parsed.get("area")
            city = parsed.get("city")

            # If any of the parsed data is missing, skip and ask for more clarification
            cat_clean = preprocess_text(cat)
            area_clean = preprocess_text(area)
            city_clean = preprocess_text(city)

            # Preprocess not_field set
            not_field_clean = {preprocess_text(x) for x in not_field}

            # Now check
            if (not cat_clean or cat_clean in not_field_clean or
                not area_clean or area_clean in not_field_clean or
                not city_clean or city_clean in not_field_clean):

                missing_data = []
                if not cat_clean or cat_clean in not_field_clean:
                    missing_data.append("Category")
                if not area_clean or area_clean in not_field_clean:
                    missing_data.append("Area")
                if not city_clean or city_clean in not_field_clean:
                    missing_data.append("City")
                missing_str = ', '.join(missing_data)
                response = f"We are unable to get your {missing_str}, Please specify it with city."
                user_input = input(f"Assitant : {response}\nUser : ")
                history_conversation[response] = user_input
                cnt += 1
                continue   # Re-run the loop for further input
            # Assuming these functions return dataframes with cosine similarity values
            df_cat_res = search_categories(cat, top_k=5)
            area_query = f"{area} {city}"
            # print(area_query)
            df_area_res = search_areas(area_query, top_k=5)

            # Check if DataFrames are empty or invalid
            if not df_cat_res.empty and not df_area_res.empty:
                threshold_cat = df_cat_res['cosine_similarity'].max()
                threshold_area = df_area_res['cosine_similarity'].max()

            print(f"Gpt Extracted:\nCategory: {cat}\nArea: {area}\nCity: {city}")

            if threshold_cat < 0.7 or threshold_area < 0.75:
                # Assuming prompt_fun4 is designed to generate a clarification question
                # print('prompt2')
                prompt_question = prompt2(history_conversation, cat, area_query, threshold_cat, threshold_area, df_cat_res['categroy_name'], df_area_res[['main_area', 'city']])
                response = get_response(prompt_question)
                cnt += 1
                user = input(f"Assitant : {response}\nUser : ")
                history_conversation[response] = user
                continue
            else:
                categories = df_cat_res[df_cat_res['cosine_similarity'] == df_cat_res['cosine_similarity'].max()]['categroy_name'].iloc[0]
                national_catid = df_cat_res[df_cat_res['cosine_similarity'] == df_cat_res['cosine_similarity'].max()]['national_catid'].iloc[0]
                area_main= df_area_res[df_area_res['cosine_similarity'] == df_area_res['cosine_similarity'].max()]['main_area'].iloc[0]
                city_name= df_area_res[df_area_res['cosine_similarity'] == df_area_res['cosine_similarity'].max()]['city'].iloc[0]
                pincode = df_area_res[df_area_res['cosine_similarity'] == df_area_res['cosine_similarity'].max()]['pincode'].iloc[0]
                output = f"""
The most similar category with the national_catid matching the user’s query:
national_catid : {national_catid}
category_name : {categories}

The most relevant areas based on the information provided by the user:
Main_area : {area_main}
Pincode : {pincode}
City : {city}
                """
                print(output)
                # return df_cat[['national_catid', 'category_name']], df_area[df_area['city'].str.lower() == city.lower()][['main_area', 'city', 'cosine_similarity']], (cat, area, city, threshold_cat, threshold_area)
                return national_catid , categories , city_name , pincode , area_main
        else:
            # Handle the case where parsing fails
            user_input = input(f"Assitant : {response}\n")
            history_conversation[response] = user_input
            cnt += 1
            continue
        cnt += 1
    # return df_cat[['national_catid', 'category_name']], df_area[df_area['city'].str.lower() == city.lower()][['main_area', 'city', 'cosine_similarity']], (cat, area, city, threshold_cat, threshold_area)
    categories = df_cat_res[df_cat_res['cosine_similarity'] == df_cat_res['cosine_similarity'].max()]['categroy_name'].iloc[0]
    national_catid = df_cat_res[df_cat_res['cosine_similarity'] == df_cat_res['cosine_similarity'].max()]['national_catid'].iloc[0]
    area_main= df_area_res[df_area_res['cosine_similarity'] == df_area_res['cosine_similarity'].max()]['main_area'].iloc[0]
    city_name= df_area_res[df_area_res['cosine_similarity'] == df_area_res['cosine_similarity'].max()]['city'].iloc[0]
    pincode = df_area_res[df_area_res['cosine_similarity'] == df_area_res['cosine_similarity'].max()]['pincode'].iloc[0]
    output = f"""
The most similar category with the national_catid matching the user’s query:
national_catid : {national_catid}
category_name : {categories}

The most relevant areas based on the information provided by the user:
Main_area : {area_main}
Pincode : {pincode}
City : {city}
    """
    print(output)
    # return df_cat[['national_catid', 'category_name']], df_area[df_area['city'].str.lower() == city.lower()][['main_area', 'city', 'cosine_similarity']], (cat, area, city, threshold_cat, threshold_area)
    return national_catid , categories, city_name , pincode , area_main

In [20]:
# step 13
# to continue the conversation with user
def cat_link(category: str, df):
    # Convert both sides to lowercase for comparison
    category_lower = category.lower()
    df['category_name_lower'] = df['category_name'].str.lower()

    # Check if the category exists
    if category_lower in df['category_name_lower'].values:
        # Find the corresponding immidate
        link_cat = df.loc[df['category_name_lower'] == category_lower, 'immidate'].iloc[0]

        # Get all categories with the same immidate
        categories = df.loc[df['immidate'] == link_cat, 'category_name'].tolist()

        return categories, category
    else:
        return [], 'END'  # No match found


In [21]:
import requests

def extract_business_info(api, limit=4):
    try:
        response = requests.get(api, timeout=60)  # disable SSL verify
        response.raise_for_status()
        api_response = response.json()
    except requests.exceptions.Timeout:
        print("⏳ API request timed out.")
        return []
    except requests.exceptions.RequestException as e:
        print(f"⚠️ API request failed: {e}")
        return []

    organic_res = api_response.get("organic_res", {})
    if not isinstance(organic_res, dict):
        return []

    results = []
    for i, (docid, details) in enumerate(organic_res.items()):
        if i >= limit:
            break
        if not isinstance(details, dict):
            continue

        company = details.get("companyname", "")
        city = details.get("city", "")
        area = details.get("area", "")
        rating = details.get("d_rating" , "")
        mobile = details.get("mobile_display", [])
        if isinstance(mobile, (int, float, str)): mobile = [str(mobile)] 
        elif not isinstance(mobile, list): mobile = []
        results.append({
            "docid": docid,
            "companyname": company,
            "mobile_display": mobile,
            "rating" : rating,
            "area": area,
            "city": city
        })

    return results


In [28]:
def clean_text1(x):
    if pd.isna(x):
        return ""
    x = x.lower()
    x = re.sub(r"[^\w]", "", x)  # keep only alphanumeric + underscore
    return x

In [5]:
import pandas as pd

def format_output(data):
    df = pd.DataFrame(data)

    # Ensure required columns exist
    required = ['companyname', 'mobile_display', 'rating', 'area', 'city']
    for c in required:
        if c not in df.columns:
            df[c] = ""

    df = df[required].copy()

    # Normalize mobile_display to a single number (first one if comma-separated)
    def pick_first_mobile(x):
        if isinstance(x, list):
            # take first non-empty element; also split if that element has commas
            for item in x:
                s = str(item).strip()
                if s:
                    return s.split(",")[0].strip()
            return ""
        if pd.isna(x):
            return ""
        return str(x).split(",")[0].strip()

    df['mobile_display'] = df['mobile_display'].apply(pick_first_mobile)
    return df


In [34]:
def search_api(api):
    data = extract_business_info(api, limit=5)
    df = format_output(data)

    # Clean headers & basic formatting
    df = df.rename(columns={
        'companyname': 'Company',
        'mobile_display': 'Mobile',
        'rating': 'Rating',
        'area': 'Area',
        'city': 'City'
    })[['Company', 'Mobile', 'Rating', 'Area', 'City']]

    # Format Rating (keep as string, 1 decimal if numeric)
    df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce').map(
        lambda x: f"{x:.1f}" if pd.notnull(x) else ""
    )
    return df

def pretty_print_df(df: pd.DataFrame) -> None:
    # Try nice terminal table
    try:
        from tabulate import tabulate
        print(tabulate(df, headers='keys', tablefmt='grid', showindex=False))
    except ImportError:
        # Fallback: good-looking pandas print
        with pd.option_context(
            'display.max_colwidth', 60,
            'display.width', 120,
            'display.colheader_justify', 'left'
        ):
            print(df.to_string(index=False))


In [1]:
# step 14
# This final function which handle the starting and ending of conversation
def whatshapp_chat():
  end = False
  # Change the path of link categories.
  df = pd.read_excel('Documents/Whatshapp_bot_final/link_categories.xlsx')
  response = 'How can I help you ?'
  area = ''
  city = ''
  start=False
  while end!=True:
    try:
      national_catid , cat_name, city_name , pincode , main_area = main_function(response , area , city , start)
      start = True
      city = city_name
      area = main_area
      city_name = clean_text1(city_name)
      national_catid = clean_text1(national_catid) 
      api = f'http://192.168.8.12:9001/web_services/CategorySearch_universal.php?catid=&city={city_name}&area={pincode}&start=1&end=10&random1=0.61266&random2=0.34577&random3=0.00138&wap_flag=0&nearme_flag=0&nearme_latitude=&nearme_longitude=&catname=&search_option=1&product_id=&catname_search=&json=1&sort_order=&national_catid={national_catid}&onational_catid=&attribute_values=&state=&darea_flg=0&wap=2&login_mobile=9008008546&pdocid=&trace='  
      data = search_api(api)     # assumes `api` is defined
      pretty_print_df(data)
      temp = input(f"Do you want to end the conversation ? (yes/no): \n").lower()
      if temp == "yes":
        end = True
        break
      categories , cat = cat_link(cat_name , df)
      if cat == 'END':
        print(f'No link categories found with {cat_name}')
        end = True
        break
      prompt2 = prompt_follow1(categories, cat)
      response = get_response(prompt2)
    except Exception as e:
      print(f"An error occurred: {e}")
      end = True

In [4]:
whatshapp_chat()

FileNotFoundError: [Errno 2] No such file or directory: 'Documents/Whatshapp_bot_final/link_categories.xlsx'

In [47]:
# To start the chatbot run this cel
whatshapp_chat()


Assitant : How can I help you ?
User :  My sister is getting marreid, I need people who can preapre food for the wedding
Assitant : Could you please specify the city where you need the food preparation service for the wedding?
 Delhi
Assitant : Could you please specify which area in Delhi are you searching for the food preparation service for the wedding?
 greater kailas


Gpt Extracted:
Category: Catering Services (where food is prepared for events)
Area: Greater Kailash
City: Delhi

The most similar category with the national_catid matching the user’s query:
national_catid : 10083351
category_name : Caterers For Events

The most relevant areas based on the information provided by the user:
Main_area : Greater Kailash
Pincode : 110048
City : Delhi
                
+------------------------+------------------+----------+---------------+--------+
| Company                | Mobile           |   Rating | Area          | City   |
+========================+==================+==========+===============+========+
| Rtw Tikki Wala         | +(91)-9999189540 |      4.2 | Paschim Vihar | Delhi  |
+------------------------+------------------+----------+---------------+--------+
| Standard Caterers      | +(91)-9312404879 |      4   | Paschim Vihar | Delhi  |
+------------------------+------------------+----------+---------------+--------+
| Om Sai Tent & Caterers |

Do you want to end the conversation ? (yes/no): 
 yes


In [53]:
whatshapp_chat()

Assitant : How can I help you ?
User :  i want ot go to naintial with my family for a vacation, suggest some agents who can help me book my trip
Assitant : Are you looking for a train ticket agent, bus booking, or flight booking?
 yes
Assitant : Could you please specify the city from which you want to travel to Nainital?
 bagalore
Assitant : We are unable to get your Area, Please specify it with city.
User :  peenya


Gpt Extracted:
Category: Travel Booking Agent (for planning and booking trips)
Area: Peenya
City: Bangalore

The most similar category with the national_catid matching the user’s query:
national_catid : 10496380
category_name : Travel Agents

The most relevant areas based on the information provided by the user:
Main_area : Peenya
Pincode : 560058
City : Bangalore
                
+--------------------------------+------------------+----------+--------+-----------+
| Company                        | Mobile           |   Rating | Area   | City      |
+================================+==================+==========+========+===========+
| Krishna Priya Tours & Travels  | +(91)-9290909992 |      5   |        | Bangalore |
+--------------------------------+------------------+----------+--------+-----------+
| Tour And Travels               | +(91)-9632244562 |      4.8 |        | Bangalore |
+--------------------------------+------------------+----------+--------+-----------+
| Sathya Sai T

Do you want to end the conversation ? (yes/no): 
 yes


In [59]:
search_categories(' Catering ')

,national_catid,categroy_name,cosine_similarity
6253,10083352,Caterers For Function,0.897012
7396,10268601,Institutes For Catering,0.896209
1398,11664893,Biryani Caterers,0.892204
5163,10083536,Catering Kitchen Equipment Manufacturers,0.889224
7987,11664517,Paan Caterers,0.888467


In [ ]:
# main_chat()

In [ ]:
# main_chat()

How can I help you?
I am looking for gym which is open for 24 hr
Could you please specify the city where you need this service?
bangalore
We are unable to get your Area, Please specify it with city.
mg road
Extracted:
Category: 24 Hour Gym Facility
Area: mg road
City: bangalore
Threshold Cat: 0.8868585824966431
Threshold Area: 0.9246256351470947

The most similar category with the national_catid matching the user’s query:
national_catid : 10208976
category_name : Fitness Centres

The most relevant areas based on the information provided by the user:
Main_area : M G Road
Pincode : 560001
City : Bangalore



In [ ]:
# print(Telegram_bot2())

1. You asked: how can i help you ?
   User replied: mujhe mere sandas ka nall theek karwana hai, paani dheeme dheeme aa raha hai. maza nahi aaya.

1. You asked: how can i help you ?
   User replied: mujhe mere sandas ka nall theek karwana hai, paani dheeme dheeme aa raha hai. maza nahi aaya.
2. You asked: The extracted category from your query is: "Plumbing Service (where plumbing issues like leaks and slow drainage are fixed)".

Could you please specify the city where you need this service?
   User replied: yamuna paar

1. You asked: how can i help you ?
   User replied: mujhe mere sandas ka nall theek karwana hai, paani dheeme dheeme aa raha hai. maza nahi aaya.
2. You asked: The extracted category from your query is: "Plumbing Service (where plumbing issues like leaks and slow drainage are fixed)".

Could you please specify the city where you need this service?
   User replied: yamuna paar
3. You asked: Could you please confirm the area in Yamuna Paar where you need this plumbing se

In [ ]:
# print(Telegram_bot2())

How can I help you?
bhuk lag rahi hai
1. You asked: how can i help you ?
   User replied: bhuk lag rahi hai

Could you please specify the city where you need this service?
delhi
1. You asked: how can i help you ?
   User replied: bhuk lag rahi hai
2. You asked: Could you please specify the city where you need this service?
   User replied: delhi

Are you looking for a restaurant, street food, or delivery service?
jaldi jo mile
1. You asked: how can i help you ?
   User replied: bhuk lag rahi hai
2. You asked: Could you please specify the city where you need this service?
   User replied: delhi
3. You asked: Are you looking for a restaurant, street food, or delivery service?
   User replied: jaldi jo mile

Extracted:
Category: Food Delivery Service
Area: N/A
City: Delhi
Threshold Cat: 0.9153370261192322
Threshold Area: 0.9073697328567505

The most similar category with the national_catid matching the user’s query:
national_catid : 11960441
category_name : Fast Food Delivery Services

Th